# Basis of singular velocity with added Fourier sequence

We need to adjust the Fourier velocity sequence to be zero at $z=0$, to maintain this property for the maximum wave:

$$w_n=1-\exp(-inz)$$

The corresponding potential sequence is:

$$\chi_n=\int_0^zw_ndz=\frac in(1-\exp(-inz)+z=\frac inw_n+z$$

The singular parts are:

$$w_0=\sqrt{1-\exp(-iz)}$$

$$\chi_0=2i\left[w_0+\frac12\ln\frac{1-w_0}{1+w_0}\right]$$

The square root singularity requires $\sum C_0=\pm1$ when we do the linear problem of finding coefficients that give $P=0$. We can thus replace the zero row by 0, except for $A_{00}=1$, to enforce this, via $\sum_nA_{0n}C_n=C_0=\pm1=B_0$.

The rest of the matrix problem is then $AC=B$ with $A_{kn}=Q_{kn}=\psi_n(z_k)=\Im\chi_n(z_k),B_k=0$ for $k>1$.

In [52]:
import numpy as np

from scipy.special import comb

from empirical import Ye

def w_n(z,N):
    '''Evaluate w_n=exp(-inz)*sqrt(1-exp(-iz)) for n in range(N)'''
    z=np.array([z])
    ez=np.exp(-1j*z)
    wN=1-np.power.outer(ez,np.arange(N))
    wN[...,0]=-np.sqrt(wN[...,1])
    return wN[0]

def chi_0(Z):
    z=np.array([Z])
    ez=np.exp(-1j*z)
    xi=np.sqrt(1-ez)
    return 2j*(xi+np.log((1-xi)/(1+xi))/2)[0]

def chi_n(Z,N):
    '''Evaluate chi_n from z=0 (see Fuchs.md)'''
    Z=np.array([Z])
    wN=w_n(Z,N)
    chiN=wN@np.diag([1,*(1j/np.arange(1,N))])+Z.reshape((*Z.shape,1))
    w0=wN[...,0]
    chiN[...,0]=2j*(w0+np.log((1-w0)/(1+w0))/2)
    return chiN[0]

def C_n(Z):
    CHI=chi_n(Z[1:],len(Z))
    zerorow=np.zeros(len(Z))
    zerorow[0]=1
    a=np.array([zerorow,*CHI.imag])
    b=np.zeros(len(Z))
    b[0]=1
    return np.linalg.solve(a,b)

In [53]:
N=5
X=np.linspace(0,np.pi,N)
Z=X+1j*Ye(X)

In [54]:
w_n(Z,3).shape,chi_n(Z,3).shape

((5, 3), (5, 3))

In [55]:
w_n(-1j,10),chi_n(-1j,10)

(array([0.7950601 +0.j, 0.63212056+0.j, 0.86466472+0.j, 0.95021293+0.j,
        0.98168436+0.j, 0.99326205+0.j, 0.99752125+0.j, 0.99908812+0.j,
        0.99966454+0.j, 0.99987659+0.j]),
 array([-0.-0.57995681j,  0.-0.36787944j,  0.-0.56766764j,  0.-0.68326236j,
         0.-0.75457891j,  0.-0.80134759j,  0.-0.83374646j,  0.-0.85727313j,
         0.-0.87504193j,  0.-0.8889026j ]))

In [56]:
h=1e-2
z=1+1j
w_n(z,5),(chi_n(z+h/2,5)-chi_n(z-h/2,5))/h

(array([ 0.96596761 +1.18397101j, -0.46869394 +2.28735529j,
         4.07493232 +6.7188497j , 20.88453084 +2.83447113j,
        36.68773248-41.32001618j]),
 array([ 0.96596634 +1.18396964j, -0.46868782 +2.28734576j,
         4.07488107 +6.71873772j, 20.88378518 +2.83436484j,
        36.68535335-41.31726157j]))